## ETL-III

En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [10]:
import pandas as pd
import requests
from datetime import datetime, timedelta

In [23]:
class Extraccion_Energia:
    def __init__(self):
        self = self

    def llamada_años(self, año1, año2):
        self.año1 = año1
        self.año2 = año2

        df_final = pd.DataFrame()
        df_renov = pd.DataFrame()
        df_no_renov = pd.DataFrame()
        for año in range(año1,año2):
            url = f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year'
            response = requests.get(url = url)

            renovables = pd.DataFrame(response.json()['included'][0]['attributes']['values'])
            df_renov = pd.concat([df_renov, renovables],  axis=0, ignore_index = True)
            df_renov['type'] = 'renovable'
        
        
            no_renovables = pd.DataFrame(response.json()['included'][1]['attributes']['values'])
            df_no_renov = pd.concat([df_no_renov, no_renovables],  axis=0, ignore_index = True)
            df_no_renov['type'] = 'no renovable'

        df_final = pd.concat([df_renov, df_no_renov],  axis=0, ignore_index = True)
        return df_final
    
    def llamada_ccaa(self, año, id_ccaa):
        self.año = año
        self.id_ccaa = id_ccaa
        
        df_final = pd.DataFrame()
        renovables = pd.DataFrame()
        no_renovables = pd.DataFrame()

        url = f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year&geo_limit=ccaa&geo_ids={id_ccaa}'
        response_ccaa = requests.get(url = url)
        
        renovables = pd.DataFrame(response_ccaa.json()['included'][0]['attributes']['values'])
        renovables['type'] = 'renovable'
        no_renovables = pd.DataFrame(response_ccaa.json()['included'][1]['attributes']['values'])
        no_renovables['type'] = 'no renovable'

        df_final = pd.concat([renovables, no_renovables],  axis=0, ignore_index = True)
        df_final['CCAA'] = id_ccaa

        return df_final
    
    def borrar_columnas(self, df, col):
        self.df = df
        self.col = col
        df.drop(columns = col, axis = 1, inplace = True)
        return df

    def limpiar_energia(self, df):
        self.df = df
        #esta parte de la función redondea las columnas numéricas
        for col in df.columns:
            if df[col].dtype == float:
                df[col] = round(df[col],2)
            else:
                pass
        return df
    
    def cambiar_fecha(self, df, col):
        self.df = df
        self.col = col

        #esta función modifica la fecha al formato solicitado y la deja con el tipo datetime
        df[col] = ((df[col].apply(pd.to_datetime)).dt.strftime("%Y-%m-%d")).apply(pd.to_datetime)
        return df

In [24]:
api = Extraccion_Energia()

In [13]:
df_años = api.llamada_años(2011, 2014)

In [20]:
df_años

,value,percentage,datetime,type
0,8.666705e+07,0.31,2011-01-01T00:00:00.000+01:00,renovable
1,8.533872e+07,0.30,2012-01-01T00:00:00.000+01:00,renovable
2,1.097577e+08,0.40,2013-01-01T00:00:00.000+01:00,renovable
3,1.926868e+08,0.69,2011-01-01T00:00:00.000+01:00,no renovable
4,1.977799e+08,0.70,2012-01-01T00:00:00.000+01:00,no renovable
5,1.635286e+08,0.60,2013-01-01T00:00:00.000+01:00,no renovable


In [26]:
api.limpiar_energia(df_años)

,value,percentage,datetime,type
0,8.666705e+07,0.31,2011-01-01,renovable
1,8.533872e+07,0.30,2012-01-01,renovable
2,1.097577e+08,0.40,2013-01-01,renovable
3,1.926868e+08,0.69,2011-01-01,no renovable
4,1.977799e+08,0.70,2012-01-01,no renovable
5,1.635286e+08,0.60,2013-01-01,no renovable


In [25]:
api.cambiar_fecha(df_años, 'datetime')

,value,percentage,datetime,type
0,8.666705e+07,0.31,2011-01-01,renovable
1,8.533872e+07,0.30,2012-01-01,renovable
2,1.097577e+08,0.40,2013-01-01,renovable
3,1.926868e+08,0.69,2011-01-01,no renovable
4,1.977799e+08,0.70,2012-01-01,no renovable
5,1.635286e+08,0.60,2013-01-01,no renovable


In [35]:
cod_comunidades2 = {'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [32]:
df_ccaa_final = pd.DataFrame()

In [33]:
for año in range(2011,2023):
    for k, v in cod_comunidades2.items(): 
        df = api.llamada_ccaa(año, v)
        df_ccaa_final = pd.concat([df_ccaa_final, df],  axis=0, ignore_index = True)

In [34]:
df_ccaa_final

,value,percentage,datetime,type,CCAA
0,1.099692e+07,0.284857,2011-01-01T00:00:00.000+01:00,renovable,4
1,2.760811e+07,0.715143,2011-01-01T00:00:00.000+01:00,no renovable,4
2,7.005216e+06,0.386497,2011-01-01T00:00:00.000+01:00,renovable,5
3,1.111968e+07,0.613503,2011-01-01T00:00:00.000+01:00,no renovable,5
4,3.385157e+05,0.170045,2011-01-01T00:00:00.000+01:00,renovable,6
...,...,...,...,...,...
355,7.891030e+06,0.352032,2022-01-01T00:00:00.000+01:00,no renovable,17
356,2.670446e+06,0.229165,2022-01-01T00:00:00.000+01:00,renovable,21
357,8.982495e+06,0.770835,2022-01-01T00:00:00.000+01:00,no renovable,21
358,1.106679e+06,0.476786,2022-01-01T00:00:00.000+01:00,renovable,20
